In [1]:
import sys
sys.path.append('./tools/experiment')

import experiment
import executor
import generator

import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
flags = {
    "cpu": {
        "cpu":1,
        "cpu-load": 10,
    },
    "mem": {
        "memrate": 1,
        "memrate-bytes": "1G",
    }
}

stress_exec =  executor.StressExecutor(**{
    "workersudo": "",
    "run_cmd": {
        "cpu": "podman run -d --rm --name stress_cpu --cpuset-cpus 120-123 localhost/my-stress-ng:v0.1",
        "mem": "podman run -d --rm --name stress_mem --cpuset-cpus 40-43 localhost/my-stress-ng:v0.1",
    },
    "stop_cmd": "podman stop",
    "opt_interval": 0,
})


with stress_exec as se:
    se.exec(**flags)

DEBUG:root:stress_executor: prologue
DEBUG:root:stress_executor: exec {'cpu': {'cpu': 1, 'cpu-load': 10}, 'mem': {'memrate': 1, 'memrate-bytes': '1G'}}
INFO:root:run:  podman run -d --rm --name stress_cpu --cpuset-cpus 120-123 localhost/my-stress-ng:v0.1 --cpu 1 --cpu-load 10
INFO:root:run:  podman run -d --rm --name stress_mem --cpuset-cpus 40-43 localhost/my-stress-ng:v0.1 --memrate 1 --memrate-bytes 1G
DEBUG:root:stress_executor: epilogue
INFO:root:run:  podman stop stress_cpu
INFO:root:run:  podman stop stress_mem


In [3]:
workload_run_cmd = generator.Workload(**{
    "base_name": "redis",
    # "base_cmd": "docker exec -it redis-client-1 memtier_benchmark",
    "base_cmd": "echo",
}
).with_flag(
    "-s", ["envoy"]
).with_flag(
    "--test-time",[180]
).with_flag(
    "-t", range(2, 10, 2)
).build()     

workload_exec = executor.WorkloadExecutor(**{
    "run_cmd": workload_run_cmd,
    # "warmup_cmd": "docker exec -it redis-client-1 memtier_benchmark -s envoy --test-time 10", 
    "warmup_cmd": "echo hello",
    "opt_interval": 0
})

with workload_exec as we:
    we.exec(**{"workload": we.workloads()[0]})
workload_run_cmd

DEBUG:root:workload_executor: prologue
DEBUG:root:workload_executor: exec {'workload': 'redis_0'}
INFO:root:run: echo hello
INFO:root:run: echo -s envoy --test-time 180 -t 2
DEBUG:root:workload_executor: epilogue


{'redis_0': 'echo -s envoy --test-time 180 -t 2',
 'redis_1': 'echo -s envoy --test-time 180 -t 4',
 'redis_2': 'echo -s envoy --test-time 180 -t 6',
 'redis_3': 'echo -s envoy --test-time 180 -t 8'}

In [4]:
exp = experiment.Experiment(
    **{
       "mode": "standard",
       "date_format": experiment.DATE_FORMAT_TIMESTAMP,
    }
    ).with_mem_stress(
        memrate_range = range(1, 16 + 1),
    ).with_cpu_stress(
        cpu_range = [1, 2, 3 ,4],
        cpuload_range = [25, 50, 75, 100],
    ).with_workload(workloads=workload_exec.workloads())


exp.run(stress_exec=stress_exec, workload_exec=workload_exec, interval=0)
exp.__dict__

INFO:root:1696921519 experiment start
DEBUG:root:stress_executor: prologue
DEBUG:root:stress_executor: exec {'mem': {'memrate': 1, 'memrate-bytes': '1G'}, 'cpu': {'cpu': 1, 'cpu-load': 25}}
INFO:root:run:  podman run -d --rm --name stress_mem --cpuset-cpus 40-43 localhost/my-stress-ng:v0.1 --memrate 1 --memrate-bytes 1G
INFO:root:run:  podman run -d --rm --name stress_cpu --cpuset-cpus 120-123 localhost/my-stress-ng:v0.1 --cpu 1 --cpu-load 25
DEBUG:root:workload_executor: prologue
DEBUG:root:workload_executor: exec {'mode': 'standard', 'workload': 'redis_0', 'stress': {'mem': {'memrate': 1, 'memrate-bytes': '1G'}, 'cpu': {'cpu': 1, 'cpu-load': 25}}}
INFO:root:run: echo hello
INFO:root:run: echo -s envoy --test-time 180 -t 2
DEBUG:root:workload_executor: exec {'mode': 'standard', 'workload': 'redis_1', 'stress': {'mem': {'memrate': 1, 'memrate-bytes': '1G'}, 'cpu': {'cpu': 1, 'cpu-load': 25}}}
INFO:root:run: echo hello
INFO:root:run: echo -s envoy --test-time 180 -t 4
DEBUG:root:workloa

{'mode': 'standard',
 'date_format': 'timestamp',
 'start_time': 1696921519,
 'end_time': 1696921519,
 'n_epoch': 16,
 'setting_per_epoch': [{'stress': {'mem': {'memrate': 1,
     'memrate-bytes': '1G'},
    'cpu': {'cpu': 1, 'cpu-load': 25}}},
  {'stress': {'mem': {'memrate': 2, 'memrate-bytes': '1G'},
    'cpu': {'cpu': 1, 'cpu-load': 50}}},
  {'stress': {'mem': {'memrate': 3, 'memrate-bytes': '1G'},
    'cpu': {'cpu': 1, 'cpu-load': 75}}},
  {'stress': {'mem': {'memrate': 4, 'memrate-bytes': '1G'},
    'cpu': {'cpu': 1, 'cpu-load': 100}}},
  {'stress': {'mem': {'memrate': 5, 'memrate-bytes': '1G'},
    'cpu': {'cpu': 2, 'cpu-load': 25}}},
  {'stress': {'mem': {'memrate': 6, 'memrate-bytes': '1G'},
    'cpu': {'cpu': 2, 'cpu-load': 50}}},
  {'stress': {'mem': {'memrate': 7, 'memrate-bytes': '1G'},
    'cpu': {'cpu': 2, 'cpu-load': 75}}},
  {'stress': {'mem': {'memrate': 8, 'memrate-bytes': '1G'},
    'cpu': {'cpu': 2, 'cpu-load': 100}}},
  {'stress': {'mem': {'memrate': 9, 'memrate-b

In [5]:
workload_exec.__dict__

{'name': 'workload_executor',
 'opt_interval': 0,
 'info_per_workload': {'redis_0': {'run_cmd': 'echo -s envoy --test-time 180 -t 2',
   'info_per_epoch': [{'addition': {},
     'start_time': 1696921519,
     'end_time': 1696921519},
    {'addition': {'stress': {'mem': {'memrate': 1, 'memrate-bytes': '1G'},
       'cpu': {'cpu': 1, 'cpu-load': 25}}},
     'start_time': 1696921519,
     'end_time': 1696921519},
    {'addition': {'stress': {'mem': {'memrate': 2, 'memrate-bytes': '1G'},
       'cpu': {'cpu': 1, 'cpu-load': 50}}},
     'start_time': 1696921519,
     'end_time': 1696921519},
    {'addition': {'stress': {'mem': {'memrate': 3, 'memrate-bytes': '1G'},
       'cpu': {'cpu': 1, 'cpu-load': 75}}},
     'start_time': 1696921519,
     'end_time': 1696921519},
    {'addition': {'stress': {'mem': {'memrate': 4, 'memrate-bytes': '1G'},
       'cpu': {'cpu': 1, 'cpu-load': 100}}},
     'start_time': 1696921519,
     'end_time': 1696921519},
    {'addition': {'stress': {'mem': {'memrate

In [6]:
dummy_stress_exec = executor.DummyExecutor(**{
    "name": "stress",
    "opt_interval": 0,
})


exp = experiment.Experiment(
    **{
       "mode": "standard",
       "date_format": experiment.DATE_FORMAT_TIMESTAMP,
    }
    ).with_epoch(
        range = range(1, 10),
    ).with_workload(workloads=workload_exec.workloads())


exp.run(stress_exec=dummy_stress_exec, workload_exec=workload_exec, interval=0)

INFO:root:1696921519 experiment start
DEBUG:root:stress: prologue
DEBUG:root:stress: exec 
DEBUG:root:workload_executor: prologue
DEBUG:root:workload_executor: exec {'mode': 'standard', 'workload': 'redis_0', 'stress': {}}
INFO:root:run: echo hello
INFO:root:run: echo -s envoy --test-time 180 -t 2
DEBUG:root:workload_executor: exec {'mode': 'standard', 'workload': 'redis_1', 'stress': {}}
INFO:root:run: echo hello
INFO:root:run: echo -s envoy --test-time 180 -t 4
DEBUG:root:workload_executor: exec {'mode': 'standard', 'workload': 'redis_2', 'stress': {}}
INFO:root:run: echo hello
INFO:root:run: echo -s envoy --test-time 180 -t 6
DEBUG:root:workload_executor: exec {'mode': 'standard', 'workload': 'redis_3', 'stress': {}}
INFO:root:run: echo hello
INFO:root:run: echo -s envoy --test-time 180 -t 8
DEBUG:root:workload_executor: epilogue
DEBUG:root:stress: epilogue
DEBUG:root:stress: prologue
DEBUG:root:stress: exec 
DEBUG:root:workload_executor: prologue
DEBUG:root:workload_executor: exec 